# Notebook Overview
This notebook provides a detailed walkthrough of the dual-stage data extraction process used to build the LeekWars Competitive Meta dataset. It leverages Python’s `requests` library to interface with the LeekWars REST API and uses `pandas` to transform deeply nested JSON action arrays into a flattened, tabular format. The scripts specifically handle complex game-state logic, such as mapping numeric equipment IDs to human-readable names and categorizing combat logs into intent-based actions.

***Note: These scripts require a valid LeekWars session token to execute. The outputs of these scripts are already provided in the attached dataset.***

# Notebook Goal
The primary objective is to demonstrate a reproducible data engineering pipeline that converts live game API responses into a structured format suitable for machine learning. By separating the process into **Entity Profiling** (extracting build stats) and **Combat Log Harvesting** (capturing turn-by-turn sequences), this notebook ensures that the relationship between a Leek's hardware and its tactical performance is preserved for future behavioral analysis and win-prediction modeling.

### **Strategic Case Studies: From Logs to Logic**
This API serves as the foundation for the **Project Alpha Strike** ecosystem. The datasets generated here were used to develop the following comprehensive meta-analyses:

* 📘 [**Stage 1: Elite Meta Decomposition**](https://www.kaggle.com/code/josephnehrenz/project-alpha-strike-p1-elite-meta-decomposition)
    * **Focus:** Cross-referencing the top 50 Leek builds to identify dominant weapon and chip archetypes in the current high-level meta.
* 📊 [**Stage 2: Tactical Execution & Probability Path Analysis**](https://www.kaggle.com/code/josephnehrenz/project-alpha-strike-p2-the-tactical-frontier)
    * **Focus:** Transforming 1.2M raw battle logs into a turn-by-turn predictive engine using Random Forest and SHAP interpretability.

### **Dataset Preview: What this API Produces**
The following is the schema generated by the extraction scripts below. This structure is optimized for time-series analysis and machine learning.

#### **1. Top Build Meta (top_builds.csv)**
*Identifies the "Hardware" of the elite meta.*

| Column | Example | Description |
| :--- | :--- | :--- |
| **leek_name** | SushiSolo | Name of the elite entity |
| **weapon_names** | `['flame_thrower', 'gazor']` | Equipped weapon templates |
| **total_life** | 3675 | Base HP stat before battle modifiers |

#### **2. Tactical Battle Logs (leek_ai.csv)**
*Captures the "Play-by-Play" decision logic.*

| Column | Example | Description |
| :--- | :--- | :--- |
| **narrative** | "PépiteDOr uses adrenaline (1 TP)" | Human-readable action summary |
| **hp_perc** | 100.0 | Current health percentage at time of action |
| **is_winner** | 1 | Target label for win-prediction models |

### **Quick Start: API Implementation Guide**
The code below is designed to respect Leek Wars API rate limits while maximizing data throughput for high-dimensional modeling.

1. **Authentication:** Set your credentials in the login prompts.
2. **Entity Profiling:** Harvests build-specific stats (Stage 1).
3. **Log Harvesting:** Flattens nested JSON into "Decision/Result" tabular pairs (Stage 2).

In [1]:
import pandas as pd
import os

# Data Audit: Validating the attached Project Alpha Strike dataset
paths = {
    "Leek Builds": "/kaggle/input/leek-wars-top-50-ai-builds-and-battle-logs/top_builds.csv",
    "Battle Logs": "/kaggle/input/leek-wars-top-50-ai-builds-and-battle-logs/leek_ai.csv"
}

for name, path in paths.items():
    if os.path.exists(path):
        print(f"✅ Found {name} at: {path}")
        df_sample = pd.read_csv(path, nrows=3)
        
        # Displaying a clean subset for the reader
        if "Builds" in name:
            display(df_sample[['rank', 'leek_name', 'level', 'weapon_names']].style.set_caption(f"Sample: {name}"))
        else:
            display(df_sample[['turn', 'actor_name', 'narrative', 'hp_perc']].style.set_caption(f"Sample: {name}"))
    else:
        print(f"❌ Missing {name}. Ensure the dataset is attached to the notebook.")

✅ Found Leek Builds at: /kaggle/input/leek-wars-top-50-ai-builds-and-battle-logs/top_builds.csv


,rank,leek_name,level,weapon_names
0,1,SushiSolo,301,"['flame_thrower', 'gazor', 'unbridled_gazor']"
1,2,Kraton,301,"['sword', 'flame_thrower', 'mysterious_electrisor', 'gazor']"
2,3,Kraneur,301,"['sword', 'flame_thrower', 'gazor', 'unbridled_gazor']"


✅ Found Battle Logs at: /kaggle/input/leek-wars-top-50-ai-builds-and-battle-logs/leek_ai.csv


,turn,actor_name,narrative,hp_perc
0,0,PépiteDOr,PépiteDOr uses adrenaline (1 TP),100.000000
1,0,PépiteDOr,N/A gains Raw Buff TP (5),100.000000
2,0,PépiteDOr,PépiteDOr uses knowledge (5 TP),100.000000


# Technical Challenges
* **Complex JSON Normalization:** The LeekWars battle logs are delivered as deeply nested JSON action arrays. A significant challenge involved flattening these hierarchical structures into a "Turn-by-Turn" tabular format while maintaining the relationship between actors, targets, and cumulative game-state effects.

* **API Rate Limiting & Session Stability:** To ensure a comprehensive harvest of the Top 50 meta without triggering server-side throttling, the pipeline implements modular extraction. This allowed for incremental data collection and error-handling for intermittent session timeouts during long-running battle-log retrievals.

* **Numeric-to-Semantic Mapping:** The raw API responses use internal numeric constants for game objects (Chips, Weapons, and Effects). The pipeline cross-references these against the game's internal manifest (`chunk-common.js`) to ensure the final CSVs are human-readable and ML-ready without further decoding.

* **State-at-Time Tracking:** Capturing the precise health (HP) and resource (TP/MP) levels at the exact moment of a decision required precise indexing of "Result" actions following each "Decision" action, ensuring the model sees the game state as the AI script saw it.

* **Version Control & Reproducibility:** To maintain a clear audit trail of the data collection logic, the source scripts are version-controlled on GitHub. This allows for modular updates to the collection pipeline as the LeekWars API evolves.

## Stage 1 - Leek Metadata Extraction

```python
# Source for Component Data: https://leekwars.com/js/chunk-common.a603e63c.js
# Gather top 50 builds
import requests
import pandas as pd
import os
import time

print("LEEKWARS BUILD HARVESTER")
print("=" * 70)

class LeekWarsCollector:
    COMPONENT_MAP = {
        290: "core", 291: "core2", 292: "core3", 293: "battery",
        294: "iron_plate", 295: "amazonite_plate", 296: "obsidian_plate",
        297: "spring", 298: "copper_spring", 299: "elinvar_spring",
        300: "ssd", 301: "nuclear_core", 302: "fan", 303: "sdcard",
        304: "cd", 305: "neural_core", 306: "neural_core_pro",
        307: "power_supply", 308: "chiyembekezo", 309: "uzoma",
        310: "kirabo", 311: "limbani", 312: "thokozani", 313: "ram",
        314: "ram2", 315: "ram3", 316: "motherboard", 317: "propulsor",
        318: "propulsor2", 319: "morus", 320: "hylocereus", 321: "apple",
        322: "nephelium", 323: "blue_mango", 324: "watercooling",
        365: "strawberry", 366: "chestnut", 369: "blue_plum",
        370: "kiwi", 371: "quince", 372: "onion", 373: "orange",
        374: "soursop", 375: "hokajin", 376: "pear", 381: "motherboard2",
        382: "motherboard3", 383: "switch", 384: "switch2", 385: "rgb",
        406: "recovery_core", 407: "recovery_ram"
    }

    def __init__(self):
        self.session = requests.Session()
        self.token = None
        self.builds_data = []
        self.weapons_by_template = {}
        self.chips_by_id = {}
        self.weapon_item_to_template = {}
        
    def login(self, username, password):
        login_url = "https://leekwars.com/api/farmer/login-token/"
        response = self.session.post(login_url, data={'login': username, 'password': password})
        if response.status_code == 200:
            data = response.json()
            self.token = data.get('token')
            self.session.headers.update({'Authorization': f'Bearer {self.token}'})
            print(f"Auth Successful: {data.get('farmer', {}).get('name')}")
            return True
        print("Login Failed.")
        return False

    def load_library(self):
        print("Syncing Item Definitions...")
        c_res = self.session.get("https://leekwars.com/api/chip/get-all")
        if c_res.status_code == 200:
            all_chips = c_res.json().get('chips', {})
            for _, chip_data in all_chips.items():
                if isinstance(chip_data, dict):
                    self.chips_by_id[int(chip_data.get('id'))] = chip_data

        w_res = self.session.get("https://leekwars.com/api/weapon/get-all")
        if w_res.status_code == 200:
            all_weapons = w_res.json().get('weapons', {})
            for _, weapon_data in all_weapons.items():
                if isinstance(weapon_data, dict):
                    tmpl, item = weapon_data.get('template'), weapon_data.get('item')
                    if tmpl: self.weapons_by_template[int(tmpl)] = weapon_data
                    if item and tmpl: self.weapon_item_to_template[int(item)] = int(tmpl)

    def collect_data(self, count=50):
        res = self.session.get("https://leekwars.com/api/ranking/get-active/leek/talent/1/null")
        ranking = res.json().get('ranking', [])[:count]
        self.load_library()
        
        print(f"\nHarvesting Top {len(ranking)} Builds...")
        print(f"{'Rank':<5} | {'Leek Name':<20} | {'Level':<5} | {'Hardware Build'}")
        print("-" * 75)

        for i, entry in enumerate(ranking):
            details = self.session.get(f"https://leekwars.com/api/leek/get/{entry['id']}").json()
            if not details: continue

            # Weapons
            w_ids = [self.weapon_item_to_template.get(int(w['template']), int(w['template'])) for w in details.get('weapons', [])]
            w_names = [self.weapons_by_template.get(wid, {}).get('name', f"ID:{wid}") for wid in w_ids]

            # Chips
            c_ids = [int(c['template']) for c in details.get('chips', [])]
            c_names = [self.chips_by_id.get(cid, {}).get('name', f"ID:{cid}") for cid in c_ids]

            # Components
            comp_ids = [int(comp['template']) for comp in details.get('components', [])]
            comp_names = [self.COMPONENT_MAP.get(cid, f"ID:{cid}") for cid in comp_ids]

            # Organizing record in a smart order for CSV analysis
            row = {
                # Identity & Meta
                'rank': entry.get('rank'),
                'leek_id': entry.get('id'),
                'leek_name': entry.get('name'),
                'level': details.get('level'),
                'talent': details.get('talent'),
                
                # Computer Stats (The "Engine")
                'cores': details.get('total_cores'),
                'ram': details.get('total_ram'),
                'frequency': details.get('total_frequency'),
                
                # Combat Stats
                'tp': details.get('tp'),
                'mp': details.get('mp'),
                'total_life': details.get('total_life'),
                'total_strength': details.get('total_strength'),
                'total_wisdom': details.get('total_wisdom'),
                'total_agility': details.get('total_agility'),
                'total_resistance': details.get('total_resistance'),
                'total_magic': details.get('total_magic'),
                'total_science': details.get('total_science'),

                # Loadout (IDs next to Names)
                'weapon_ids': w_ids,
                'weapon_names': w_names,
                'chip_ids': c_ids,
                'chip_names': c_names,
                'component_ids': comp_ids,
                'component_names': comp_names
            }
            self.builds_data.append(row)
            
            # Intuitive console log
            hw_summary = f"C:{row['cores']} R:{row['ram']} F:{row['frequency']}"
            print(f"#{row['rank']:<4} | {row['leek_name']:<20} | Lvl {row['level']:<3} | {hw_summary}")
            time.sleep(0.1)

    def save(self):
        df = pd.DataFrame(self.builds_data)
        # Reordering columns to make the CSV human-friendly
        cols = ['rank', 'leek_id', 'leek_name', 'level', 'talent', 'cores', 'ram', 'frequency', 'tp', 'mp', 
                'total_life', 'total_strength', 'total_wisdom', 'total_agility', 'total_resistance', 
                'total_magic', 'total_science', 'weapon_ids', 'weapon_names', 'chip_ids', 
                'chip_names', 'component_ids', 'component_names']
        df = df[cols]
        
        # Robust pathing: Save relative to script location
        script_dir = os.path.dirname(os.path.abspath(__file__))
        project_root = os.path.dirname(script_dir)
        refined_dir = os.path.join(project_root, "data_refined")
        
        os.makedirs(refined_dir, exist_ok=True)
        filename = os.path.join(refined_dir, 'top_builds.csv')
        df.to_csv(filename, index=False)
        print(f"\nData Exported to {filename}")

if __name__ == "__main__":
    u = input("Username: ")
    p = input("Password: ")
    
    collector = LeekWarsCollector()
    if collector.login(u, p):
        collector.collect_data(50)
        collector.save()
```

## Stage 2 - Battle Log Harvesting

```python
# Gather recent 50 solo fights for top 50 leeks
import requests
import os
import pandas as pd
import time
from leekwars_action_schema import (
    parse_action_safe, calculate_distance, get_effect_name,
    ACTION_USE_WEAPON, ACTION_USE_CHIP, ACTION_MOVE_TO, ACTION_LEEK_TURN,
    ACTION_SUMMON, ACTION_NEW_TURN, ACTION_SET_WEAPON_NEW, ACTION_SET_WEAPON,
    ACTION_LIFE_LOST, ACTION_LIFE_DAMAGE, ACTION_POISON_DAMAGE, ACTION_DAMAGE_RETURN, ACTION_NOVA_DAMAGE,
    ACTION_CARE, ACTION_NOVA_VITALITY, ACTION_BOOST_VITA, ACTION_END_TURN,
    ACTION_ADD_CHIP_EFFECT, ACTION_ADD_WEAPON_EFFECT, ACTION_PLAYER_DEAD
)

# HIGH AGENCY ACTIONS: Tactical choices that drive the game
INTERESTING_ACTIONS = {ACTION_USE_CHIP, ACTION_USE_WEAPON, ACTION_SET_WEAPON_NEW, ACTION_SET_WEAPON, ACTION_SUMMON}

class BattleStateTracker:
    def __init__(self, raw_leeks):
        self.entities = {}
        for l in raw_leeks:
            eid = int(l['id'])
            self.entities[eid] = {
                'id': eid,
                'name': l['name'],
                'team': l['team'],
                'cur_hp': int(l['life']),
                'max_hp': int(l['life']),
                'cell': l.get('cellPos'),
                'is_summon': l.get('summon', False)
            }

    def update(self, parsed):
        code = parsed['action_code']
        
        # MOVEMENT
        if code == ACTION_MOVE_TO:
            ent = self.entities.get(parsed.get('entity_id'))
            if ent: ent['cell'] = parsed.get('dest_cell')
            
        # HEALTH CHANGES
        elif code in {ACTION_LIFE_LOST, ACTION_LIFE_DAMAGE, ACTION_POISON_DAMAGE, ACTION_DAMAGE_RETURN, ACTION_NOVA_DAMAGE}:
            ent = self.entities.get(parsed.get('entity_id'))
            if ent and parsed.get('damage'):
                ent['cur_hp'] -= parsed['damage']
                if code == ACTION_NOVA_DAMAGE:
                    ent['max_hp'] -= parsed['damage']
                    
        elif code in {ACTION_CARE, ACTION_NOVA_VITALITY}:
            ent = self.entities.get(parsed.get('entity_id'))
            if ent and parsed.get('heal_amount'):
                ent['cur_hp'] = min(ent['max_hp'], ent['cur_hp'] + parsed['heal_amount'])
                
        elif code == ACTION_BOOST_VITA:
            ent = self.entities.get(parsed.get('entity_id'))
            if ent and parsed.get('vita_boost'):
                ent['max_hp'] += parsed['vita_boost']
                ent['cur_hp'] += parsed['vita_boost']
                
        # SUMMONS
        elif code == ACTION_SUMMON:
            sid = parsed.get('summon_entity_id')
            if sid and sid not in self.entities: # FIX: Avoid overwriting existing info
                self.entities[sid] = {
                    'id': sid, 'name': f"Summon_{sid}", 'team': -1,
                    'cur_hp': 0, 'max_hp': 0, 'cell': parsed.get('cell'), 'is_summon': True
                }

    def get_state(self, entity_id):
        return self.entities.get(entity_id, {})
    
    def get_entity_at(self, cell):
        if cell is None: return None
        for eid, ent in self.entities.items():
            if ent.get('cell') == cell:
                return ent
        return None

class HighAgencyBattleCollector:
    def __init__(self):
        self.session = requests.Session()
        self.fight_actions = []
        self.chips_metadata = {}
        self.weapons_metadata = {}
        self.top_leek_ids = set()
        
    def login(self, user, pw):
        r = self.session.post("https://leekwars.com/api/farmer/login-token/", 
                             data={'login': username, 'password': password})
        if r.status_code == 200 and 'token' in r.json():
            self.session.headers.update({'Authorization': f"Bearer {r.json()['token']}"})
            print(f"Logged in: {user}")
            self.load_metadata()
            return True
        print(f"Login failed")
        return False
    
    def load_metadata(self):
        print("Loading chip/weapon metadata...")
        c_res = self.session.get("https://leekwars.com/api/chip/get-all")
        if c_res.status_code == 200:
            for _, c in c_res.json().get('chips', {}).items():
                tid = int(c['template'])
                self.chips_metadata[tid] = {'name': c.get('name'), 'cost': c.get('cost')}
        
        w_res = self.session.get("https://leekwars.com/api/weapon/get-all")
        if w_res.status_code == 200:
            for _, w in w_res.json().get('weapons', {}).items():
                tid = int(w['template'])
                self.weapons_metadata[tid] = {'name': w.get('name'), 'cost': w.get('cost')}
        print(f"  Metadata loaded.")

    def load_top_leeks(self):
        path = "data_refined/top_builds.csv"
        if os.path.exists(path):
            df = pd.read_csv(path)
            self.top_leek_ids = set(df['leek_id'].astype(int))
            print(f"Loaded {len(self.top_leek_ids)} Target Leek IDs for filtering.")
            return True
        return False

    def collect(self, top_n=50, max_fights=50):
        if not self.top_leek_ids:
            ranking = self.session.get("https://leekwars.com/api/ranking/get-active/leek/talent/1/null").json().get('ranking', [])[:top_n]
            self.top_leek_ids = {int(l['id']) for l in ranking}

        total_leeks = len(self.top_leek_ids)
        for i, lid in enumerate(self.top_leek_ids, 1):
            print(f"[{i}/{total_leeks}] Collecting fights for Leek ID {lid}...", end=" ", flush=True)
            try:
                hist = self.session.get(f"https://leekwars.com/api/history/get-leek-history/{lid}").json()
                fights = [f for f in hist.get('fights', []) if f['type'] == 0][:max_fights]
                success_count = 0
                for f in fights:
                    if self._process_fight(f['id']): success_count += 1
                print(f"Done ({success_count} fights captured).")
                time.sleep(0.05)
            except Exception as e: print(f"Error: {e}")

    def _process_fight(self, fight_id):
        try:
            f_resp = self.session.get(f"https://leekwars.com/api/fight/get/{fight_id}").json()
            if not f_resp or 'data' not in f_resp: return False
            
            winner_team = f_resp.get('winner')
            data = f_resp.get('data', {})
            tracker = BattleStateTracker(data.get('leeks', []))
            
            active_id, turn = None, 0
            current_weapon = {} # actor_id -> weapon_template_id
            
            for act in data.get('actions', []):
                parsed = parse_action_safe(act)
                if parsed.get('error'): continue
                
                code = parsed['action_code']
                tracker.update(parsed) # Always update state
                
                if code == ACTION_NEW_TURN:
                    turn = parsed.get('turn', turn + 1)
                    continue
                if code == ACTION_LEEK_TURN:
                    active_id = parsed['entity_id']
                    continue
                if code in {ACTION_SET_WEAPON, ACTION_SET_WEAPON_NEW}:
                    if active_id is not None:
                        current_weapon[active_id] = parsed.get('weapon_id')
                
                # CATEGORIZATION
                category = "OTHER"
                if code in {ACTION_USE_CHIP, ACTION_USE_WEAPON, ACTION_SUMMON, ACTION_MOVE_TO}:
                    category = "DECISION"
                elif code in {ACTION_LIFE_LOST, ACTION_CARE, ACTION_BOOST_VITA, ACTION_POISON_DAMAGE, 
                            ACTION_LIFE_DAMAGE, ACTION_NOVA_DAMAGE, ACTION_ADD_CHIP_EFFECT, ACTION_ADD_WEAPON_EFFECT}:
                    category = "RESULT"
                elif code in {ACTION_SET_WEAPON, ACTION_SET_WEAPON_NEW}:
                    category = "SETUP"
                elif code == ACTION_END_TURN:
                    category = "STATUS"
                else: continue
                
                actor = tracker.get_state(active_id)
                if not actor and category != "RESULT": continue # Need an actor for decisions
                
                # ITEM & COST (Safe Access)
                item_name, tp_used = "N/A", 0
                mp_used = parsed.get('mp_used', 0)
                if code == ACTION_USE_CHIP:
                    meta = self.chips_metadata.get(parsed.get('chip_id'), {})
                    item_name = meta.get('name', f"Chip_{parsed.get('chip_id')}")
                    tp_used = meta.get('cost', 0)
                elif code == ACTION_USE_WEAPON:
                    wid = current_weapon.get(active_id)
                    meta = self.weapons_metadata.get(wid, {})
                    item_name = meta.get('name', f"Weapon_{wid}")
                    tp_used = meta.get('cost', 0)
                elif code in {ACTION_SET_WEAPON, ACTION_SET_WEAPON_NEW}:
                    wid = parsed.get('weapon_id')
                    meta = self.weapons_metadata.get(wid, {})
                    item_name = meta.get('name', f"Weapon_{wid}")
                    tp_used = 1
                elif code == ACTION_SUMMON:
                    item_name = "Summon"

                # TARGETING (Safe Access)
                target_id = parsed.get('entity_id') if category == "RESULT" else parsed.get('target_id')
                target_name, target_rel = "N/A", "N/A"
                
                t_ent = None
                if target_id is not None:
                    t_ent = tracker.get_state(target_id)
                elif parsed.get('cell') is not None:
                    t_ent = tracker.get_entity_at(parsed['cell'])
                
                if t_ent:
                    target_name = t_ent.get('name', "N/A")
                    if actor:
                        if t_ent.get('id') == active_id: target_rel = "Self"
                        elif t_ent.get('team') != actor.get('team'): target_rel = "Enemy"
                        else: target_rel = "Ally"
                elif parsed.get('cell') is not None:
                    target_name = f"Cell_{parsed['cell']}"
                    target_rel = "Location"

                # NARRATIVE LOG RECONSTRUCTION (Extremely Defensive)
                log_actor = target_name if category == "RESULT" else actor.get('name', 'N/A')
                log_verb = "N/A"
                if code == ACTION_USE_CHIP: log_verb = "uses"
                elif code == ACTION_USE_WEAPON: log_verb = "attacks with"
                elif code == ACTION_MOVE_TO: log_verb = "moves"
                elif code == ACTION_SUMMON: log_verb = "summons"
                elif code in {ACTION_SET_WEAPON, ACTION_SET_WEAPON_NEW}: log_verb = "takes weapon"
                elif code in {ACTION_LIFE_LOST, ACTION_LIFE_DAMAGE, ACTION_POISON_DAMAGE, ACTION_NOVA_DAMAGE}: log_verb = "loses"
                elif code in {ACTION_CARE, ACTION_NOVA_VITALITY}: log_verb = "gains"
                elif code == ACTION_BOOST_VITA: log_verb = "gains total health"
                elif code in {ACTION_ADD_CHIP_EFFECT, ACTION_ADD_WEAPON_EFFECT}: log_verb = "gains"
                elif code == ACTION_PLAYER_DEAD: log_verb = "is dead!"
                
                log_obj = "N/A"
                if code in {ACTION_USE_CHIP, ACTION_SET_WEAPON, ACTION_SET_WEAPON_NEW}: log_obj = item_name
                elif code == ACTION_USE_WEAPON: log_obj = item_name
                elif code == ACTION_SUMMON: log_obj = target_name
                elif code in {ACTION_LIFE_LOST, ACTION_LIFE_DAMAGE, ACTION_POISON_DAMAGE, ACTION_NOVA_DAMAGE, ACTION_CARE, ACTION_NOVA_VITALITY, ACTION_BOOST_VITA}: 
                    log_obj = "HP"
                elif code in {ACTION_ADD_CHIP_EFFECT, ACTION_ADD_WEAPON_EFFECT}:
                    from leekwars_action_schema import get_effect_name
                    log_obj = get_effect_name(parsed.get('effect_type'))
                
                log_qty = "N/A"
                if code in {ACTION_USE_CHIP, ACTION_USE_WEAPON}: log_qty = f"{tp_used} TP"
                elif code == ACTION_MOVE_TO: log_qty = f"{mp_used} MP"
                elif parsed.get('value') is not None: log_qty = str(parsed['value'])
                
                log_extra = "N/A"
                if parsed.get('is_critical'): log_extra = "... Critical hit!"
                elif parsed.get('duration'): log_extra = f"({parsed['duration']} turns)"
                
                # Full narrative string reconstruction
                parts = [str(log_actor), str(log_verb)]
                if log_obj != "N/A": parts.append(str(log_obj))
                if log_qty != "N/A": parts.append(f"({log_qty})")
                if log_extra != "N/A": parts.append(str(log_extra))
                narrative = " ".join(parts)

                self.fight_actions.append({
                    "fight_id": fight_id, "turn": turn,
                    "action_category": category, "action_name": parsed.get('action_name', 'N/A'),
                    "actor_id": active_id, "actor_name": actor.get('name', 'N/A'),
                    "target_name": target_name, "target_relation": target_rel, "item_name": item_name,
                    "hp_raw": actor.get('cur_hp', 0) if actor else 0,
                    "hp_max": actor.get('max_hp', 0) if actor else 0,
                    "hp_perc": round(actor['cur_hp'] / actor['max_hp'] * 100, 1) if actor and actor.get('max_hp', 0) > 0 else 0,
                    "tp_used": tp_used, "is_winner": 1 if actor and actor.get('team') == winner_team else 0,
                    "log_verb": log_verb, "log_object": log_obj, "log_quantity": log_qty, "log_extra": log_extra,
                    "narrative": narrative
                })
            return True
        except Exception as e:
            # print(f"Error parsing fight {fight_id}: {e}")
            return False

    def save(self):
        if not self.fight_actions: return
        df = pd.DataFrame(self.fight_actions)
        
        # Reorder columns as requested
        column_order = [
            "fight_id", "turn", "actor_id", "actor_name", "log_verb", 
            "log_object", "log_quantity", "log_extra", "narrative", 
            "action_category", "action_name", "target_name", "target_relation", 
            "item_name", "hp_raw", "hp_max", "hp_perc", "tp_used", "is_winner"
        ]
        # Ensure all columns exist before reordering
        df = df[[c for c in column_order if c in df.columns]]
        
        project_root = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
        path = os.path.join(project_root, "data_refined", "leek_ai.csv")
        df.to_csv(path, index=False)
        print(f"Saved {len(df)} lines to {path}")

if __name__ == "__main__":
    u = input("Username: ")
    p = input("Password: ")
    
    bot = HighAgencyBattleCollector()
    if bot.login(u, p):
        bot.load_top_leeks()
        bot.collect(top_n=50, max_fights=50)
        bot.save()
```